### ספריות

In [175]:
import os
import sys
import fiona
import pandas as pd
import geopandas as gpd
from openpyxl import load_workbook

In [176]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### העלת משתנים להרצת הקוד

In [177]:
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

output_location=df_inputs_outputs['location'][2]

create_forecast_basic_software_location = os.path.dirname(software_folder_location)

sys.path.append(create_forecast_basic_software_location)

create_forecast_basic_software_df = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(create_forecast_basic_software_location))

TAZ_V4_date = '240408'

### פונקציות גלובליות

In [178]:
from global_functions import up_load_shp, up_load_df, drop_geo, make_point, up_load_gdb, change_cols_names,add_geo_info_gdb,add_geo_info_shp

ממשיך את הקוד עכשיו שיכלול גם את המאפיינים הגיאוגרפים שעיסא צריך

In [179]:
col=['Taz_num','geometry']

In [180]:
taz_border=up_load_shp(r'{}\background_files\TAZ_V4_{}_with_geo_info.shp'.format(create_forecast_basic_software_location, TAZ_V4_date))[col]

In [181]:
#העלה של נתוני אנשים התפלגות גילים שנוצר לפני הקוד הזה
path=r'{}\Intermediates'.format(software_folder_location)
taz=up_load_df(path,'taz_before_add_geo')

In [182]:
taz['tazSector']=1 #ערבי
taz.loc[taz['main_secto']=='U_Orthodox','tazSector']=2
taz.loc[taz['main_secto']=='Jewish','tazSector']=3
taz.loc[taz['main_secto']=='Palestinian','tazSector']=4

לייצר פומה

In [183]:
poly_pumas=up_load_shp(r'W:\Data\Forecast\Tools\forecast_git\create_forecast_basic\background_files\poly_pumas.shp')
col_old=['poly_puma',  'F3', 'F2', 'F1', 'geometry']
col_new=['poly_puma',  '3', '2', '1', 'geometry']
poly_pumas=drop_geo(change_cols_names(poly_pumas,col_old,col_new))
pumas_by_poly_sector=poly_pumas.melt(id_vars='poly_puma',var_name='tazSector',value_name='PUMA')
pumas_by_poly_sector['tazSector']=pumas_by_poly_sector['tazSector'].astype(int)

In [184]:
taz=taz.merge(pumas_by_poly_sector,on=['poly_puma','tazSector'],how='left')

In [185]:
taz.loc[taz['PUMA']==0,'PUMA']=999
taz.loc[taz['pop']==0,'PUMA']=999
taz.loc[taz['main_secto']=='Palestinian','PUMA']=999
taz.loc[taz['jeru_metro']==0,'PUMA']=999

פלט של ההיברדי

In [186]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [187]:

col_needed=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed',
'slope',
'Urban']

col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed',
'slop',
'urban']

In [188]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [189]:
df=change_cols_names(taz, col_needed, col_new_name)

In [190]:
col_to_int=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed',
'slop',
'urban']

In [191]:
for c in col_to_int:
    df.loc[:, c] = df.loc[:, c].astype(int)

In [192]:
df.sort_values(by='TAZ',ascending=True).to_excel(r'{}\BaseProjections2020_{}.xlsx'.format(output_location,file_date),index=False)

DISTRICT

In [193]:
taz['DISTRICT']=999
taz.loc[taz['jew']==0,'DISTRICT']=1
taz.loc[(taz['main_secto']=='U_Orthodox')&(taz['in_jerusal']=='yes'),'DISTRICT']=2
taz.loc[(taz['main_secto']=='Jewish')&(taz['in_jerusal']=='yes'),'DISTRICT']=3
taz.loc[(taz['main_secto']=='Jewish')&(taz['in_jerusal']=='no')&(taz['jeru_metro']==1),'DISTRICT']=5
taz.loc[(taz['main_secto']=='U_Orthodox')&(taz['in_jerusal']=='no')&(taz['jeru_metro']==1),'DISTRICT']=6
taz.loc[taz['pop']==0,'DISTRICT']=999

In [194]:

col_needed=['Taz_num','Agg_taz_nu','PUMA','DISTRICT','REGION','SCHOOLDIST']

col_new_name=['TAZ','AGG_TAZ','PUMA','DISTRICT','REGION','SCHOOLDISTRICT']

In [195]:
change_cols_names(taz, col_needed, col_new_name).sort_values(by='TAZ',ascending=True).to_excel(r'{}\puma2020_{}.xlsx'.format(output_location,file_date),index=False)

SED

In [196]:
col_needed=['Taz_num',
 'Taz_num',
 'hh',
 'PUMA',
 'DISTRICT',
 'county',
 'area',
 'parktot',
 'majunivenr',
 'tazSector',
 'Indus',
 'Com_hotel',
 'Business',
 'Public',
 'emp_Education',
 'agri',
 'total_emp',
 'UOA1',
 'UOA2',
 'UOA3',
 'SEA1',
 'SEA2',
 'SEA3',
 'ARA1',
 'ARA2',
 'ARA3',
 'TOA1',
 'TOA2',
 'TOA3',
 'Univ_AR',
 'Univ_SE',
 'UNIVENRORTHMALE',
 'UNIVENRORTHFEMALE',
 'ieold',
 'superZone',
 'IEProp',
 'Taz1',
 'perScaled',
 'EIProp',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCon',
 'PaidBuffer',
 'Rest_EmpBu',
 'FreeBuffer',
 'SCHOOLDIST',
 'SCHOOLDIST',
 'highBusine',
 'searchtime',
 'walktime',
 'cost']

In [197]:
col_new_name=['maz',
 'taz',
 'hh_total',
 'puma',
 'district',
 'county',
 'area',
 'parktot',
 'majunivenr',
 'tazSector',
 'Indus',
 'Com_hotel',
 'Off_Bsness',
 'Public',
 'Education',
 'Agri',
 'totemp',
 'UOA1',
 'UOA2',
 'UOA3',
 'SEA1',
 'SEA2',
 'SEA3',
 'ARA1',
 'ARA2',
 'ARA3',
 'TOA1',
 'TOA2',
 'TOA3',
 'UNIVENRARAB',
 'UNIVENRSEC',
 'UNIVENRORTHMALE',
 'UNIVENRORTHFEMALE',
 'ieold',
 'superZone',
 'IEProp',
 'Taz1',
 'perScaled',
 'EIProp',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCons',
 'PaidBuffer',
 'Rest_EmpBuffer',
 'FreeBuffer',
 'schDistrict',
 'schDistrictAgg',
 'highBusinessFlag',
 'searchtime',
 'walktime',
 'cost']

In [199]:
change_cols_names(taz, col_needed, col_new_name).sort_values(by='maz',ascending=True).to_excel(r'{}\SED_2020_{}.xlsx'.format(output_location,file_date),index=False)

In [200]:
taz.to_excel(r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(output_location,file_date),index=False)

In [201]:
#load excel file
workbook = load_workbook(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))

#open workbook
sheet = workbook.active

#modify the desired cell
sheet["B4"] = forecast_version_folder_location
sheet["B5"] = False
sheet["B6"] = r'{}\background_files\TAZ_V4_{}_with_geo_info.shp'.format(create_forecast_basic_software_location, TAZ_V4_date)

#save the file
workbook.save(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))